TODO: debud this shit

1. Squares might be overlapping

In [ ]:
import gc
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [ ]:
warnings.filterwarnings('ignore')

### Create Shape

In [ ]:
from st_forecasting.utils.geo import create_polygons

In [ ]:
# not the settings that were used initially

setup = {
    'lat_min': 24.4,# 24.322995193443617,
    'lon_min': 46.4, # 46.31937651439429,
    'lat_max': 25.102463220881223,
    'lon_max': 47.08501151812489
}

In [ ]:
squares = create_polygons(
    setup['lat_min'],
    setup['lon_min'],
    4000,
    36//2
)

In [ ]:
squares.geometry.iloc[-1].wkt

In [ ]:
# Starting point is left low

squares.explore()

In [ ]:
squares.to_parquet('data/riyadh_squares.pq')

In [ ]:
n = int(np.sqrt(len(squares)))

ids_mat = np.rot90(squares.square_id.values.reshape(n, n))

In [ ]:
ids_mat

In [ ]:
with open('data/riyadh_squares.npy', 'wb') as file:
    np.save(file, ids_mat)

In [ ]:
pd_squares = pd.DataFrame({'square_id': squares.square_id.values})

In [ ]:
pd_squares.to_parquet('data/pd_squares.pq')

### Load Data

Warning: this takes a lot of time!

In [ ]:
from pyhive import presto

from st_forecasting.utils.geo import preprocess
from queries import get_locations_data

In [ ]:
conn = presto.connect(
    host='presto-python-r-script-cluster.careem-engineering.com',
    username='presto_python_r',
    port=8080
)

In [ ]:
date_range = [str(x).split(' ')[0] for x in pd.date_range(start='2023-05-01', end='2023-05-01', freq='D')]  # start='2022-09-01'

len(date_range)

In [ ]:
df_list = []
intents_list = []


for date in tqdm(date_range):
    sub = pd.read_sql(sql=get_locations_data(date, 3), con=conn)
    intents_list.append(sub)
    sub = preprocess(sub, squares, setup)
    df_list.append(sub)

intents_df = pd.concat(intents_list, ignore_index=True)
snapshots = pd.concat(df_list, ignore_index=True).sort_values(['ts', 'square_id']).reset_index(drop=True)


del sub
del df_list
del intents_list

_ = gc.collect()

In [ ]:
len(snapshots)

In [ ]:
snapshots.head(20)

In [ ]:
intents_df.to_parquet('data/riyadh_data_med.pq')
snapshots.to_parquet('data/riyadh_snap_med.pq')